In [12]:
#%matplotlib inline
#%reload_ext signature

import numpy as np
import dicom
import os
import matplotlib.pyplot as plt
from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
from plotly.graph_objs import *
init_notebook_mode(connected=True) 

import matplotlib.image as img
WD = os.getcwd()
print(WD)

/home/silvio/Dokumente/GitHub/BrainMRT


In [14]:
series = [g for g in glob(WD + "/DICOM/ST000000/*")]
series

['/home/silvio/Dokumente/GitHub/BrainMRT/DICOM/ST000000/SE000002',
 '/home/silvio/Dokumente/GitHub/BrainMRT/DICOM/ST000000/SE000000',
 '/home/silvio/Dokumente/GitHub/BrainMRT/DICOM/ST000000/SE000001',
 '/home/silvio/Dokumente/GitHub/BrainMRT/DICOM/ST000000/SE000004',
 '/home/silvio/Dokumente/GitHub/BrainMRT/DICOM/ST000000/SE000006',
 '/home/silvio/Dokumente/GitHub/BrainMRT/DICOM/ST000000/SE000009',
 '/home/silvio/Dokumente/GitHub/BrainMRT/DICOM/ST000000/SE000013',
 '/home/silvio/Dokumente/GitHub/BrainMRT/DICOM/ST000000/SE000018',
 '/home/silvio/Dokumente/GitHub/BrainMRT/DICOM/ST000000/SE000003',
 '/home/silvio/Dokumente/GitHub/BrainMRT/DICOM/ST000000/SE000008',
 '/home/silvio/Dokumente/GitHub/BrainMRT/DICOM/ST000000/SE000014',
 '/home/silvio/Dokumente/GitHub/BrainMRT/DICOM/ST000000/SE000015',
 '/home/silvio/Dokumente/GitHub/BrainMRT/DICOM/ST000000/SE000016',
 '/home/silvio/Dokumente/GitHub/BrainMRT/DICOM/ST000000/SE000007',
 '/home/silvio/Dokumente/GitHub/BrainMRT/DICOM/ST000000/SE0000

In [101]:
def load_scan(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 1
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    #intercept = -1024
    #slope = 1
    #intercept = scans[0].RescaleIntercept
    #slope = scans[0].RescaleSlope
    
    #if slope != 1:
    #    image = slope * image.astype(np.float64)
    #    image = image.astype(np.int16)
        
    #image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)


outputPath = "./data/"

series = [g for g in glob(WD + "/DICOM/ST000000/*")]
series.sort()

for i in list([1,2,3,4,5,6,7,8,9,10,11,12,13,14]):
    id=0
    patient = load_scan(series[i])
    imgs = get_pixels_hu(patient)

    np.save(outputPath + "fullImages/"+"fullimages_%d_%d.npy" % (id, i), imgs)
    
    
    

In [102]:
def load_scan(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

outputPath = "./data/"

def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
#    spacing = map(float, ([scan[0].SliceThickness] + scan[0].PixelSpacing))
    spacing = map(float, ([2.5] + scan[0].PixelSpacing)) #guessed
    spacing = np.array(list(spacing))

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor)
    
    return image, new_spacing

id = 0
for i in list([1,2,3,4,5,6,7,8,9,10,11,12,13,14]):
    images = np.load(outputPath+"fullImages/"+'fullimages_{}_{}.npy'.format(id,i))
    print("Shape before resampling\t", images.shape)
    images_resamp, spacing = resample(images, patient, [1,1,1])
    print ("Shape after resampling\t", images_resamp.shape)
    np.save(outputPath +"resampledImages/"+ "resampled_images_%d_%d.npy" % (id, i), images_resamp)



Shape before resampling	 (5, 162, 162)
Shape after resampling	 (12, 42, 42)
Shape before resampling	 (3, 162, 162)
Shape after resampling	 (8, 42, 42)
Shape before resampling	 (3, 162, 162)
Shape after resampling	 (8, 42, 42)
Shape before resampling	 (27, 512, 512)
Shape after resampling	 (68, 133, 133)
Shape before resampling	 (25, 1098, 1098)
Shape after resampling	 (62, 286, 286)
Shape before resampling	 (71, 122, 122)
Shape after resampling	 (178, 32, 32)
Shape before resampling	 (71, 122, 122)
Shape after resampling	 (178, 32, 32)
Shape before resampling	 (36, 256, 256)
Shape after resampling	 (90, 67, 67)
Shape before resampling	 (39, 384, 384)
Shape after resampling	 (98, 100, 100)
Shape before resampling	 (192, 256, 256)
Shape after resampling	 (480, 67, 67)
Shape before resampling	 (170, 256, 256)
Shape after resampling	 (425, 67, 67)
Shape before resampling	 (170, 256, 256)
Shape after resampling	 (425, 67, 67)
Shape before resampling	 (26, 288, 288)
Shape after resampling	 (